In [1]:
! pip install --q unstructured langchain
# ! pip install --"unstructured[all-docs]"

^C
ERROR: Operation cancelled by user


# Ingesting PDF

In [1]:
from pprint import pprint
from langchain_community.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain_community.document_loaders import PyPDFLoader

In [2]:
local_path = "/mnt/c/Users/Green.PC/Documents/ASAMA/project/rag-chatbot-telegram-data/data/WEF_The_Global_Cooperation_Barometer_2024.pdf"

if local_path:
    loader = PyPDFLoader(file_path=local_path)
    data = loader.load_and_split()
else:
    print('Upload a PDF file')

In [3]:
pprint(data[0].page_content)

('The Global Cooperation \n'
 'Barometer 2024\n'
 'INSIGHT REPORT\n'
 'JANUARY 2024In collaboration with \n'
 'McKinsey & Company')


# Vector Embeddings

**"ollama serve" in terminal**

In [4]:
# # !docker exec -it ollama ollama pull nomic-embed-text
# !docker exec -it ollama ollama list
# !ollama pull nomic-embed-text
# !ollama list

NAME                   	ID          	SIZE  	MODIFIED     
nomic-embed-text:latest	0a109f422b47	274 MB	15 hours ago	
llama3:latest          	365c0bd3c000	4.7 GB	42 hours ago	
zsh:1: command not found: ollama
zsh:1: command not found: ollama


In [5]:
# !pip install --q chromadb
# !pip install --q langchain-text-splitters

^C
ERROR: Operation cancelled by user


In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [7]:
# Split and chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [8]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    collection_name='local-rag'
    
)

OllamaEmbeddings: 100%|██████████| 30/30 [00:19<00:00,  1.55it/s]


# Retrieval

In [9]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [10]:
# LLM from Ollama
local_model = "llama3"
llm = ChatOllama(model=local_model)

In [11]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [12]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question:{question}"""

prompt = ChatPromptTemplate.from_template(template)

In [13]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
chain.invoke(input(""))

In [14]:
chain.invoke("What are the 5 pillars of global cooperation?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.60it/s]


'There is no information provided in this context about the 5 pillars of global cooperation. The document appears to be a collection of pages from a report, with each page containing text and metadata. The question cannot be answered based on this context alone.'